# **Phishing Domain Detection (Data Collection & Feature Extraction)**

### **Objective :** Collect data and extract necessary features from that data to train Machine Learning models

# **1.0] Data Collection**

We'll use this kaggle dataset : https://www.kaggle.com/siddharthkumar25/malicious-and-benign-urls.

It contains 450k domain url's out of which 345k are legitimate & 104k are malicious.

From this dataset 10,000 url's are randomly collected from each class to train ML models.

In [56]:
# Check if GPU is being used.

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [57]:
import pandas as pd

# Loading the downloaded dataset from kaggle 
df = pd.read_csv("/content/urldata.csv")
df.head(10)

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0
5,5,https://www.reddit.com,benign,0
6,6,https://www.yahoo.com,benign,0
7,7,https://www.google.co.in,benign,0
8,8,https://www.qq.com,benign,0
9,9,https://www.amazon.com,benign,0


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  450176 non-null  int64 
 1   url         450176 non-null  object
 2   label       450176 non-null  object
 3   result      450176 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ MB


In [59]:
df.shape

(450176, 4)

In [60]:
# Printing number of legit and fraud domain urls
df["label"].value_counts()

benign       345738
malicious    104438
Name: label, dtype: int64

# **1.1] Collect Phishing Urls**

We'll randomly collect 10,000 phishing domain urls and save it as another csv file

In [61]:
# Filtering out rows with phishing urls
phishing_df = df[df["result"] == 1]
phishing_df.head(10)

,Unnamed: 0,url,label,result
345738,345738,http://atualizacaodedados.online,malicious,1
345739,345739,http://webmasteradmin.ukit.me/,malicious,1
345740,345740,http://stcdxmt.bigperl.in/klxtv/apps/uk/,malicious,1
345741,345741,https://tubuh-syarikat.com/plugins/fields/files/,malicious,1
345742,345742,http://rolyborgesmd.com/exceword/excel.php?.ra...,malicious,1
345743,345743,http://ongelezen-voda.000webhostapp.com/inlogg...,malicious,1
345744,345744,http://www.valenzaceramic.com/home/webapps/e52...,malicious,1
345745,345745,http://membership-issue.forteimpex.com/dk2mmm=...,malicious,1
345746,345746,http://membership-issue.forteimpex.com/dk2mmm=...,malicious,1
345747,345747,http://chronopost-service-enligne.net/56123s/r...,malicious,1


In [62]:
# removing unecessary columns
phishing_df = phishing_df[["url","label","result"]]
phishing_df.head()

,url,label,result
345738,http://atualizacaodedados.online,malicious,1
345739,http://webmasteradmin.ukit.me/,malicious,1
345740,http://stcdxmt.bigperl.in/klxtv/apps/uk/,malicious,1
345741,https://tubuh-syarikat.com/plugins/fields/files/,malicious,1
345742,http://rolyborgesmd.com/exceword/excel.php?.ra...,malicious,1


In [63]:
phishing_df.shape

(104438, 3)

In [64]:
# Randomly collecting 10,000 samples and saving as csv file
phishing_df = phishing_df.sample(n=10000)
phishing_df.reset_index(inplace=True,drop=True) # IMPORTANT TO DO
print(phishing_df.info())

phishing_df.to_csv("phishing_urls.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     10000 non-null  object
 1   label   10000 non-null  object
 2   result  10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB
None


# **1.2] Collect Legitimate Urls**

We'll randomly collect 10,000 legitimate domain urls and save it as another csv file

In [65]:
# Filtering out rows with phishing urls
legitimate_df = df[df["result"] == 0]
legitimate_df.head(10)

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0
5,5,https://www.reddit.com,benign,0
6,6,https://www.yahoo.com,benign,0
7,7,https://www.google.co.in,benign,0
8,8,https://www.qq.com,benign,0
9,9,https://www.amazon.com,benign,0


In [66]:
# removing unecessary columns

legitimate_df = legitimate_df[["url","label","result"]]
legitimate_df.head()

,url,label,result
0,https://www.google.com,benign,0
1,https://www.youtube.com,benign,0
2,https://www.facebook.com,benign,0
3,https://www.baidu.com,benign,0
4,https://www.wikipedia.org,benign,0


In [67]:
legitimate_df.shape

(345738, 3)

In [68]:
# Randomly collecting 10,000 samples and saving as csv file
legitimate_df = legitimate_df.sample(n=10000)
legitimate_df.reset_index(inplace=True,drop=True) # IMPORTANT TO DO
print(legitimate_df.info())

legitimate_df.to_csv("legitimate_urls.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     10000 non-null  object
 1   label   10000 non-null  object
 2   result  10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB
None


# **2.0] Feature Extraction**

In this step, features are extracted from the URLs dataset.

The extracted features are categorized into

1. Address Bar based Features
2. Domain based Features
3. HTML & Javascript based Features

## **2.1] Address Bar Based Features :**

Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.

- Domain of URL
- IP Address in URL
- "@" Symbol in URL
- Length of URL
- Depth of URL
- Redirection "//" in URL
- "http/https" in Domain name
- Using URL Shortening Services “TinyURL”
- Prefix or Suffix "-" in Domain

Each of these features are explained and the coded below:

In [69]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

### **2.1.1] Domain of the URL**
Here, we are just extracting the domain present in the URL. This feature doesn't have much significance in the training. May even be dropped while training the model.

In [70]:
# get Domain of the URL (Domain) 
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain


### **2.1.2] IP Address in the URL**
Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

In [71]:
# Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

i = havingIP(url="www.deepeshmhatre.com")
print(i)

0


### **2.1.3] "@" Symbol in URL**
Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [72]:
# Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

i = haveAtSign(url="www.deepeshmhatre.com")
print(i)

0


### **2.1.4. Length of URL**
Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.

If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [73]:
# Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

i = getLength(url="www.deepeshmhatre.com")
print(i)

0


### **2.1.5] Depth of URL**
Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

In [74]:
# Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

i = getDepth(url="www.tesla.com")
print(i)

1


### **2.1.6. Redirection "//" in URL**
Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [75]:
# Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

i = redirection(url="www.tesla.com")
print(i)

0


### **2.1.7. "http/https" in Domain name**
Checks for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.

If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [76]:

# Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

### **2.1.8] Using URL Shortening Services “TinyURL”**
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [77]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [78]:
# Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

### **2.1.9. Prefix or Suffix "-" in Domain**
Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [79]:
# Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

## **2.2] Domain Based Features :**

Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

- DNS Record
- Website Traffic
- Age of Domain
- End Period of Domain

Each of these features are explained and the coded below:

In [80]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
!pip install python-whois
import whois
import urllib
import urllib.request
from datetime import datetime

### **2.2.1] DNS Record**
For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname. If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [81]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

### **2.2.2] Web Traffic**
This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [82]:
# Web traffic (Web_Traffic)
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

i = web_traffic(url="www.deepesh.com")
print(i)

1


### **2.2.3] Age of Domain**
This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but difference between creation and expiration time.

If age of domain < 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [83]:
# Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age


### **2.2.4] End Period of Domain**
This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less for this project.

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).


In [84]:
# End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

## **2.3] HTML and JavaScript based Features**
Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

- IFrame Redirection
- Status Bar Customization
- Disabling Right Click
- Website Forwarding

Each of these features are explained and the coded below:

In [85]:

# importing required packages for this section
import requests

### **2.3.1] IFrame Redirection**
IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation.

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [86]:

# IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

### **2.3.2] Status Bar Customization**
Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the “onMouseOver” event, and check if it makes any changes on the status bar

If the response is empty or onmouseover is found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [87]:
# Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

### **2.3.3] Disabling Right Click**
Phishers use JavaScript to disable the right-click function, so that users cannot view and save the webpage source code. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “event.button==2” in the webpage source code and check if the right click is disabled.

If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [88]:
# Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1


### **2.3.4] Website Forwarding**
The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times.

In [89]:
# Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

# **3.0] Computing URL Features**

Now we use all the methods that we created and will create new feature columns.

Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

In [90]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  # HTML & Javascript based features (4)
  try:
    response = requests.get(url)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features


# testing the function
features = featureExtraction(url="https://www.spacex.com/",label=0)
features

['spacex.com', 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]

### **NOTE : Check the Indexes of dataframes,if they are not reseted during sampling,then reset them now using reset_index() function.**

In [91]:
legitimate_df.head()

,url,label,result
0,https://www.bef.co.uk/,benign,0
1,https://www.thestarphoenix.com/sports/Conrad+g...,benign,0
2,https://www.legacy.com/obituaries/myjournalcou...,benign,0
3,https://www.en.wikipedia.org/wiki/Bob_Riley,benign,0
4,https://www.shoushu.com/,benign,0


In [92]:
phishing_df.head()

,url,label,result
0,http://czaragroup.com/admin/css/excel/index.ph...,malicious,1
1,http://cdec-stleonard.ca/hercule/maps/portale/...,malicious,1
2,https://sites.google.com/site/approtecscures/,malicious,1
3,https://www.cilwculture.com/admin/styles/1/8dd...,malicious,1
4,http://biovinci.com.br/4j3ypm,malicious,1


# **Creating dataset for Legitimate Urls**

### **NOTE : Running this code on GPU is recommended**


In [93]:
# Check if GPU is being used.

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [94]:
from tqdm import tqdm

#Extracting the feautres & storing them in a list
legitimate_features = []
label = 0       # IMPORTANT : Label is 0 for legitimate urls

for i in tqdm(range(0, 10000)):
  url = legitimate_df["url"][i]
  legitimate_features.append(featureExtraction(url,label))

print(legitimate_features)

100%|██████████| 10/10 [02:19<00:00, 13.91s/it]

[['bef.co.uk', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], ['thestarphoenix.com', 0, 0, 1, 6, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0], ['legacy.com', 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0], ['en.wikipedia.org', 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0], ['shoushu.com', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0], ['croonet.com', 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0], ['classmates.com', 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0], ['tv-address.com', 0, 0, 1, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], ['scientificamerican.com', 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0], ['gamespot.com', 0, 0, 1, 4, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]]


In [95]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legitimate_features, columns= feature_names)

In [96]:
# Saving as csv file
legitimate.to_csv("Legitimate_Url_Features.csv")

# load the saved csv file again
legitimate = pd.read_csv("Legitimate_Url_Features.csv")
legitimate.head(10)

,Unnamed: 0,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,0,bef.co.uk,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,1,thestarphoenix.com,0,0,1,6,0,0,1,0,0,0,1,1,0,0,1,0,0
2,2,legacy.com,0,0,1,3,0,0,0,0,0,1,0,1,0,0,1,0,0
3,3,en.wikipedia.org,0,0,0,2,0,0,0,0,0,1,1,1,1,1,1,1,0
4,4,shoushu.com,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0
5,5,croonet.com,0,0,0,2,0,0,1,0,0,1,0,1,1,1,1,1,0
6,6,classmates.com,0,0,1,3,0,0,0,0,0,1,1,1,0,0,1,0,0
7,7,tv-address.com,0,0,1,3,0,0,0,1,0,0,0,0,0,0,1,0,0
8,8,scientificamerican.com,0,0,1,1,0,0,0,0,0,1,1,1,0,0,1,0,0
9,9,gamespot.com,0,0,1,4,0,0,1,0,0,1,0,1,0,0,1,1,0


# **Creating dataset for Phishing Urls**

### **NOTE : Running this code on GPU is recommended**


In [97]:
from tqdm import tqdm

#Extracting the feautres & storing them in a list
phishing_features = []
label = 1     # IMPORTANT : Label is 1 for phishing urls

for i in tqdm(range(0, 10000)):
  url = phishing_df["url"][i]
  phishing_features.append(featureExtraction(url,label))

print(phishing_features)

100%|██████████| 10/10 [00:15<00:00,  1.59s/it]

[['czaragroup.com', 0, 1, 1, 4, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1], ['cdec-stleonard.ca', 0, 0, 1, 4, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ['sites.google.com', 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1], ['cilwculture.com', 0, 0, 1, 8, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], ['biovinci.com.br', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1], ['joe12341.netii.net', 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1], ['ip-173-201-135-29.ip.secureserver.net', 0, 0, 1, 3, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1], ['lexingtonobgyn.com', 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1], ['news.crateandbarrel.com', 0, 1, 1, 3, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1], ['nofkfnfdmaklfdlb.online ', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]


In [98]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phishing_features, columns= feature_names)

In [99]:
# Saving as csv file
phishing.to_csv("Phishing_Url_Features.csv")

# load the saved csv file again
phishing = pd.read_csv("Phishing_Url_Features.csv")
phishing.head(10)

,Unnamed: 0,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,0,czaragroup.com,0,1,1,4,0,0,1,0,0,1,1,1,0,0,1,0,1
1,1,cdec-stleonard.ca,0,0,1,4,0,0,0,1,1,1,1,1,1,1,1,1,1
2,2,sites.google.com,0,0,0,2,0,0,0,0,0,1,1,1,0,0,1,0,1
3,3,cilwculture.com,0,0,1,8,0,0,0,0,1,1,1,1,1,1,1,1,1
4,4,biovinci.com.br,0,0,0,1,0,0,0,0,0,1,1,0,1,1,1,1,1
5,5,joe12341.netii.net,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1
6,6,ip-173-201-135-29.ip.secureserver.net,0,0,1,3,0,0,0,1,0,1,1,1,0,0,1,0,1
7,7,lexingtonobgyn.com,0,0,0,2,0,0,0,0,0,1,0,1,0,0,1,0,1
8,8,news.crateandbarrel.com,0,1,1,3,0,0,0,0,0,1,1,0,1,1,1,1,1
9,9,nofkfnfdmaklfdlb.online,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1


# **Final Dataset**

In the above section we formed two dataframes of legitimate & phishing URL features. Now, we will combine them to a single dataframe and export the data to csv file for the Machine Learning training done in other notebook.


In [100]:
#Concatenating the dataframes into one 
final_data = pd.concat([legitimate, phishing])
final_data.reset_index(inplace=True,drop=True)

# dropping useless column
final_data = final_data.drop("Unnamed: 0",axis=1)

final_data.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,bef.co.uk,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,thestarphoenix.com,0,0,1,6,0,0,1,0,0,0,1,1,0,0,1,0,0
2,legacy.com,0,0,1,3,0,0,0,0,0,1,0,1,0,0,1,0,0
3,en.wikipedia.org,0,0,0,2,0,0,0,0,0,1,1,1,1,1,1,1,0
4,shoushu.com,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0


In [101]:
final_data.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
15,joe12341.netii.net,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1
16,ip-173-201-135-29.ip.secureserver.net,0,0,1,3,0,0,0,1,0,1,1,1,0,0,1,0,1
17,lexingtonobgyn.com,0,0,0,2,0,0,0,0,0,1,0,1,0,0,1,0,1
18,news.crateandbarrel.com,0,1,1,3,0,0,0,0,0,1,1,0,1,1,1,1,1
19,nofkfnfdmaklfdlb.online,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1


In [102]:
# Storing and Exporting data as csv file
final_data.to_csv('Phishing_Detection.csv', index=False)

# **Conclusion**

**The objective of this notebook have been achieved,we collected data,extracted features from that data and finally created our dataset which will be used for training ML models in next notebook**